# EVALUATION NOTEBOOK

### Effect of Prefilters
The dataset was transcribed with two different prefilters.

In [ ]:
import h5py
import dataloader
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

def computeStatistics(data):
    median = np.median(data)
    iqr = np.subtract(*np.percentile(data, [75, 25]))
    return median,iqr

def getFscoreResults(filepath,alphabet="majmin",printMetadata=False):
    results = {"combined":[]}

    with h5py.File(filepath,"r") as file:
        text = "----Transcription Metadata----\n"
        for key,value in file.attrs.items():
            text += f"{key}={value}\n"
        if printMetadata:
            print(text)
        for dset in ["beatles","rwc_pop","rw","queen"]:
            f_scores = []
            for subgrp_name in file[f"{dset}/"]:
                subgrp = file[f"{dset}/{subgrp_name}"]
                if subgrp.attrs.get("name") not in dataloader.OUTLIERS:
                    f_scores.append(subgrp.attrs.get(f"{alphabet}_f"))
            results[dset] = f_scores
            results["combined"] += f_scores
    return results

def plotResults(data,xlabels,colors,yticks=np.arange(-20,25,5),ylim=(-20,10),figsize=(5.5, 2.5)):
    """creates colored violinplots for data"""
    fig, ax1 = plt.subplots(figsize=figsize)
    parts = ax1.violinplot(data,showmeans=False, showmedians=True,
            showextrema=False)
    bplot = ax1.boxplot(data,
                    showfliers=True,medianprops=dict(linestyle=None,linewidth=0),
                    flierprops=dict(markerfacecolor='k', marker='o',markersize=1),
                    widths=0.1)
    parts["cmedians"].set_color("red")
    for x,color in zip(parts["bodies"],colors):
            x.set_color(color)

    ax1.set_yticks(yticks)
    ax1.set_ylim(ylim)
    ax1.set_ylabel("Delta F-score in %")
    ax1.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
    ax1.set_xlim(0.5, len(xlabels) + 0.5);
    ax1.grid("on")
    fig.tight_layout(pad=0.1)
    return fig,ax1

def printRow(data,prefilter="RP",alphabet="majmin"):
    text = f"{prefilter} & {alphabet} &"
    for dset in ["beatles","rwc_pop","rw","queen","combined"]:
        med,iqr = computeStatistics(data[dset])
        text += f"{med:0.2f}$\\pm${iqr:0.2f} &"
    text = text[:-1]+ "\\\\"
    print(text)

rp_majmin = getFscoreResults("../results/recurrence_plot.hdf5","majmin",False)
rp_sevenths = getFscoreResults("../results/recurrence_plot.hdf5","sevenths")
median_majmin = getFscoreResults("../results/median_none.hdf5","majmin", True)
median_sevenths = getFscoreResults("../results/median_none.hdf5","sevenths")
print(f"----------Table of results------------")
rp_text = "RP &"
median_text = "Median &"
header = "prefilter & scheme &"
for dset in ["beatles","rwc_pop","rw","queen","combined"]:
    if dset == "rwc_pop":
        header += "rwc\_pop & "
    else:
        header += dset +" &"
header = header[:-1] + "\\\\"
print(header)
printRow(rp_majmin,"RP","majmin")
printRow(median_majmin,"Median","majmin")
print("\\midrule")
printRow(rp_sevenths,"RP","sevenths")
printRow(median_sevenths,"Median","sevenths")
# plot paired difference
delta_f_majmin = []
delta_f_sevenths = []
for dset in ["beatles","rwc_pop","rw","queen","combined"]:
    delta_f_majmin.append([100*(x-y) for x,y in zip(rp_majmin[dset],median_majmin[dset])])
    delta_f_sevenths.append([100*(x-y) for x,y in zip(rp_sevenths[dset],median_sevenths[dset])])
colors = ['r','g','b','purple']
xlabels = ["Beatles","RWC","RW","Queen","Combined"]
fig,ax = plotResults(delta_f_majmin,xlabels,colors,np.arange(-15,20,5),ylim=(-20,20))
fig.savefig("../latex/figures/prefilter_delta_f_majmin.png")
fig,ax = plotResults(delta_f_sevenths,xlabels,colors,np.arange(-15,20,5),ylim=(-20,20))
fig.savefig("../latex/figures/prefilter_delta_f_sevenths.png")

# wilcoxon test
print("-----Wilcoxon test for paired samples MAJMIN-------")
print("dataset & N & z & p & r \\\\")
print("\\toprule")
for i,dset in enumerate(["beatles","rwc\_pop","rw","queen","combined"]):
    res = scipy.stats.wilcoxon(delta_f_majmin[i],method='approx')
    N = len(delta_f_majmin[i])
    p = res.pvalue
    z = res.zstatistic
    r = np.abs(z) / np.sqrt(N)
    if p < 0.001:
        print(f"{dset} & {N} & {z:0.1f} & $<$.001 & {r:0.1f}\\\\")
    else:    
        print(f"{dset} & {N} & {z:0.1f} & {p:0.3f} & {r:0.1f}\\\\")

print("-----Wilcoxon test for paired samples SEVENTHS-------")
print("dataset & N & z & p & r \\\\")
print("\\toprule")
for i,dset in enumerate(["beatles","rwc\_pop","rw","queen","combined"]):
    res = scipy.stats.wilcoxon(delta_f_sevenths[i],method='approx')
    N = len(delta_f_sevenths[i])
    p = res.pvalue
    z = res.zstatistic
    r = np.abs(z) / np.sqrt(N)
    if p < 0.001:
        print(f"{dset} & {N} & {z:0.1f} & $<$.001 & {r:0.1f}\\\\")
    else:    
        print(f"{dset} & {N} & {z:0.1f} & {p:0.3f} & {r:0.1f}\\\\")


### Study on Source Separation
Vocals, drums were removed seperately with a hybrid transformer source separation network (demucs) to enhance the chroma.</br>
After seperating the drums, the vocals were again removed from the remaining signal.



In [ ]:
# source_separation_none


### NSGT-CQT  --- Tuning issues

In [ ]:
import numpy as np
import utils
import matplotlib.pyplot as plt
import dataloader
import librosa.display
import librosa
import nsgt

dset = "rwc_pop"
track_id = "RM-P002" 
time_interval = (5,15)
## Boys: 0105 -> blurry bass note  Tuning?
## Anna: 0103 
class interpolate:
    def __init__(self,cqt,Ls):
        from scipy.interpolate import interp1d
        self.intp = [interp1d(np.linspace(0, Ls, len(r)), r) for r in cqt]
    def __call__(self,x):
        try:
            len(x)
        except:
            return np.array([i(x) for i in self.intp])
        else:
            return np.array([[i(xi) for i in self.intp] for xi in x]) 
        
dataset = dataloader.Dataloader(dset,"/home/max/ET-TI/Masterarbeit/chord-recognition/mirdata","none")
audiopath, annotationpath = dataset[track_id]
gt = utils.loadChordAnnotations(annotationpath)
sig = utils.loadAudiofile(audiopath,offset=time_interval[0],duration=time_interval[1]-time_interval[0])
estimated_tuning = librosa.estimate_tuning(y=sig,sr=22050)
f0 = librosa.tuning_to_A4(estimated_tuning)
print(f"f0={f0:0.1f} Hz")
midi2hz = lambda m,f0 : f0*(2.0**((m-69.0)/12.0))
fs = 22050
octaves = 6

fmin = midi2hz(24,f0) # C2
fmax = midi2hz(24 + 12*octaves,f0) # C6
bpo = 12
cq_nsgt = nsgt.CQ_NSGT(fmin,fmax, bpo,fs, len(sig))
# forward transform 
coeffs = cq_nsgt.forward(sig)

# interpolate CQT to get a grid
x = np.arange(0, len(sig), 64)
cqt = interpolate(map(np.abs, coeffs[2:]), len(sig))(x).T#

from scipy.fftpack import dct,idct
pitchgram_energy = cqt**2
pitchgram_energy = np.log(100 * pitchgram_energy + 1);  
vLogDCT = dct(pitchgram_energy, norm='ortho', axis=0);  
vLogDCT[:33,:] = 0 
vLogDCT[33,:] = 0.5 * vLogDCT[33,:]
pitchgram_energy = idct(vLogDCT, norm='ortho', axis=0)

chroma = np.zeros((12,pitchgram_energy.shape[1]),dtype=float)
for i in range(pitchgram_energy.shape[0]):
    index = i % 12
    chroma[(i+1) % 12,:] += pitchgram_energy[i,:]
chroma = chroma / np.sum(np.abs(chroma) + np.finfo(float).eps, axis=0)
fig,ax = plt.subplots(3,1,height_ratios=(10,1,5),figsize=(9,5))
t_cqt = time_interval[0] + x / fs
cqt_db = 20 * np.log10(cqt)
pmax = np.percentile(cqt_db, 99.99)
img = librosa.display.specshow(cqt_db,
                         ax=ax[0],x_coords=t_cqt,y_axis="cqt_hz",
                         fmin=fmin,fmax=fmax,x_axis="s",
                         cmap="viridis",vmax=pmax,vmin=pmax-80)
utils.plotChordAnnotations(ax[1],gt[0],gt[1],time_interval)
utils.plotChromagram(ax[2],t_cqt,chroma);

### CQT and pitchgram analysis

In [ ]:
import librosa
import numpy as np
import utils
import matplotlib.pyplot as plt
from scipy.fftpack import dct,idct
import dataloader
import librosa.display
import IPython.display as ipd
plt.ion()
dset = "rwc_pop"
track_id = "RM-P003" 

## Boys: 0105 -> blurry bass note  Tuning?
## Anna: 0103 

dataset = dataloader.Dataloader(dset,"/home/max/ET-TI/Masterarbeit/chord-recognition/mirdata","none")
audiopath, annotationpath = dataset[track_id] 
sig = utils.loadAudiofile(audiopath,duration=20)
gt = utils.loadChordAnnotations(annotationpath)

####### tuned CQT ############
midi_start = 36
octaves = 6
cqt_params = {"sr":22050,"bins_per_octave":36,"hop_length":1024,"fmin":librosa.midi_to_hz(midi_start)}

estimated_tuning = librosa.estimate_tuning(y=sig,sr=22050,bins_per_octave=36)
C = np.abs(librosa.vqt(sig,filter_scale=1,tuning=estimated_tuning,gamma=0,n_bins=octaves*36, **cqt_params))
C_db = 20 * np.log10(C+ np.finfo(float).eps)
pmax = np.percentile(C_db, 99.99)

time = (5,15)
fig,(ax0,ax1) = plt.subplots(2,1,height_ratios=(1,10),figsize=(6,4))
## librosa
utils.plotChordAnnotations(ax0,gt[0],gt[1],time_interval=time)
img = librosa.display.specshow(C_db,
                         ax=ax1,y_axis="cqt_hz",x_axis="s",
                         vmax=pmax, vmin=pmax - 80,
                         cmap="viridis",**cqt_params)
ax1.set_xlim(time)
fig.tight_layout()

In [ ]:
## apply gaussian window function to deemphasize low frequent hum and overtones 
## centered around C4 +/- 15 note 
window=True
pitchgram_cqt = np.zeros((12*octaves,C.shape[1]),dtype=float)
pitchgram_cqt_windowed = np.zeros((12*octaves,C.shape[1]),dtype=float)
# pitchgram window function
for note in range(midi_start, midi_start+12*octaves):
    # weight and pick every third amplitude value
    pitchgram_cqt[note - midi_start,:] =  C[(note-midi_start)*3, :]
    pitchgram_cqt_windowed[note - midi_start,:] = np.exp(-(note-60)**2 / (2*15**2)) * C[(note-midi_start)*3,:] 
pitchgram_energy = pitchgram_cqt ** 2

fig,(ax0,ax1) = plt.subplots(1,2,figsize=(6,4))
pitchgram_params = cqt_params
pitchgram_params["bins_per_octave"] = 12
librosa.display.specshow(librosa.amplitude_to_db(pitchgram_cqt,ref=np.max),
                         ax=ax0,y_axis="cqt_hz",x_axis="s",cmap="viridis",**pitchgram_params)
img = librosa.display.specshow(librosa.amplitude_to_db(pitchgram_cqt_windowed,ref=np.max),
                         ax=ax1,x_axis="s",cmap="viridis",**pitchgram_params)
ax0.set_xlim(time)
ax1.set_xlim(time)
fig.colorbar(img);

In [ ]:
### Logarithmic Compression and Liftering 
eta = 100
nCRP = 33
if True:
    pitchgram_log = np.log(eta * pitchgram_energy + 1)
else:
    pitchgram_log = pitchgram_cqt
vLogDCT = dct(pitchgram_log, norm='ortho', axis=0);  
vLogDCT[:nCRP,:] = 0 
vLogDCT[nCRP,:] = 0.5 * vLogDCT[nCRP,:]
pitchgram_lift = idct(vLogDCT, norm='ortho', axis=0)

# pitch folding
crp = pitchgram_lift.reshape(octaves,12,-1) 
crp = np.sum(crp, axis=0)
crp = np.clip(crp,0,None) # clip negative values
crp = crp /np.sum(np.abs(crp) + np.finfo(float).tiny, axis=0) # apply manhattan norm

t_chroma = utils.timeVector(crp.shape[1],hop_length=cqt_params["hop_length"])

fig,((ax0,ax00),(ax1,ax11),(ax2,ax22),(ax3,ax33)) = plt.subplots(4,2,figsize=(7,7),width_ratios=(10,.1),height_ratios=(2,2,0.3,1))
img = librosa.display.specshow(librosa.amplitude_to_db(pitchgram_log,ref=np.max),
                         ax=ax0,x_axis="s",y_axis="cqt_note",cmap="viridis",**pitchgram_params)
fig.colorbar(img,cax=ax00)
ax0.set_title("log compressed pitchgram (dB)")
img = librosa.display.specshow(pitchgram_lift,
                         ax=ax1,x_axis="s",y_axis="cqt_note",cmap="viridis",**pitchgram_params,vmin=-3,vmax=3)
fig.colorbar(img,cax=ax11)
ax1.set_title("liftered pitchgram")
for ax in [ax0,ax1]:
    ax.set_xlabel("")
ax2.set_title("CRP feature with annotations")
utils.plotChordAnnotations(ax2,gt[0],gt[1],time_interval=time)
ax22.set_axis_off()
ax2.set_xlabel("")
ax2.set_xticklabels([])
img = utils.plotChromagram(ax3,t_chroma,crp)
fig.colorbar(img,cax=ax33);
fig.tight_layout()

### Transsprition Results

In [ ]:
import matplotlib.pyplot as plt
from visualization import visualizationApp
%matplotlib inline
visualizationApp("/home/max/ET-TI/Masterarbeit/chord-recognition/results");
plt.ioff(); # interactive mode off

In [ ]:
from visualization import load_results
from dataloader import OUTLIERS
from scipy import stats
import matplotlib.pyplot as plt
import utils
import numpy as np
plt.ion() # interactive mode on
median,datasets = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/median_drums.hdf5")
cpss,datasets = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/cpss.hdf5")
delta_f = [100*(x.majmin_f - y.majmin_f) for x,y in zip(cpss,median) if x.name not in OUTLIERS]
delta_f_sevenths = [100*(x.sevenths_f - y.sevenths_f) for x,y in zip(cpss,median) if x.name not in OUTLIERS]
fig, ax = plt.subplots(figsize=(3, 2.8))
utils.create_violinplot(ax,[delta_f,delta_f_sevenths],["majmin","sevenths"],"DodgerBlue")
fig.tight_layout(pad=0.1)
ax.set_yticks(np.arange(-40,50,10))
ax.set_ylim(-50,50)
ax.set_ylabel("Delta F-score in %");
res = stats.wilcoxon(delta_f,method='approx')
res_sevenths = stats.wilcoxon(delta_f_sevenths,method='approx')

print("Wilcoxon test for paired samples")
print(f"Majmin: z={res.zstatistic:0.1f}, p={res.pvalue:0.3f}, r={np.abs(res.zstatistic) / np.sqrt(len(delta_f)):0.2f}")
print(f"Sevenths: z={res_sevenths.zstatistic:0.1f}, p={res_sevenths.pvalue:0.3f}, r={np.abs(res_sevenths.zstatistic) / np.sqrt(len(delta_f)):0.2f}")


### Comparison of results between rp and median filtering and the deep chroma processor

In [ ]:
from visualization import load_results
from dataloader import OUTLIERS
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import utils

results_median,datasets = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/median_none.hdf5")
results_rp,_ = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/rp_none.hdf5")
results_dcp,_ = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/dcp_none.hdf5")

results_median = [x for x in results_median if x.name not in OUTLIERS]
results_rp = [x for x in results_rp if x.name not in OUTLIERS]
               
majmin_dcp = [100*x.majmin_f for x in results_dcp]
sevenths_dcp = [100*x.sevenths_f for x in results_dcp]

comparison_median_dcp = [(x,y) for x,y in zip(results_median,results_dcp) if x.majmin_f > y.majmin_f ]
# for data in comparison_median_dcp:
#     print(f"{data[0].name} -{data[0].track_id} - MEDIAN: {data[0].majmin_f} , DCP: {data[1].majmin_f}")

majmin_f_median = [100*x.majmin_f for x in results_median]
sevenths_f_median = [100*x.sevenths_f for x in results_median]
majmin_f_rp = [100*x.majmin_f for x in results_rp]
sevenths_f_rp = [100*x.sevenths_f for x in results_rp]

fig, ax = plt.subplots(figsize=(3, 2.8))
utils.create_violinplot(ax,[majmin_f_median,majmin_dcp],["Median","DCP"],"DodgerBlue")
fig.tight_layout(pad=0.1)
fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/median_rp_majmin_result.png")

fig, ax = plt.subplots(figsize=(3, 2.8)) 
utils.create_violinplot(ax,[sevenths_f_median,sevenths_dcp],["Median","DCP"],"DodgerBlue")
fig.tight_layout(pad=0.1)
fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/median_rp_sevenths_result.png")

majmin_f = [100*x.majmin_f for x in results_median if x.name not in OUTLIERS]
majmin_f_rp = [100*x.majmin_f for x in results_rp if x.name not in OUTLIERS]
beatles_f = [100*x.majmin_f for x in results_median if x.name not in OUTLIERS and x.dataset == "beatles"]
beatles_f_rp = [100*x.majmin_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "beatles"]
rwc_f = [100*x.majmin_f for x in results_median if x.name not in OUTLIERS and x.dataset == "rwc_pop"]
rwc_f_rp = [100*x.majmin_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "rwc_pop"]
rw_f = [100*x.majmin_f for x in results_median if x.name not in OUTLIERS and x.dataset == "rw"]
rw_f_rp = [100*x.majmin_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "rw"]
queen_f = [100*x.majmin_f for x in results_median if x.name not in OUTLIERS and x.dataset == "queen"]
queen_f_rp = [100*x.majmin_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "queen"]

majmin_f_sev = [100*x.sevenths_f for x in results_median if x.name not in OUTLIERS]
majmin_f_rp_sev = [100*x.sevenths_f for x in results_rp if x.name not in OUTLIERS]
beatles_f_sev = [100*x.sevenths_f for x in results_median if x.name not in OUTLIERS and x.dataset == "beatles"]
beatles_f_rp_sev = [100*x.sevenths_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "beatles"]
rwc_f_sev = [100*x.sevenths_f for x in results_median if x.name not in OUTLIERS and x.dataset == "rwc_pop"]
rwc_f_rp_sev = [100*x.sevenths_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "rwc_pop"]
rw_f_sev = [100*x.sevenths_f for x in results_median if x.name not in OUTLIERS and x.dataset == "rw"]
rw_f_rp_sev = [100*x.sevenths_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "rw"]
queen_f_sev = [100*x.sevenths_f for x in results_median if x.name not in OUTLIERS and x.dataset == "queen"]
queen_f_rp_sev = [100*x.sevenths_f for x in results_rp if x.name not in OUTLIERS and x.dataset == "queen"]

delta_f = [x-y for x,y in zip(majmin_f_rp,majmin_f)]
beatles_delta = [x-y for x,y in zip(beatles_f_rp,beatles_f)]
rwc_delta = [x-y for x,y in zip(rwc_f_rp,rwc_f)]
rw_delta = [x-y for x,y in zip(rw_f_rp,rw_f)]
queen_delta = [x-y for x,y in zip(queen_f_rp,queen_f)]
data = [delta_f,beatles_delta,rwc_delta,rw_delta,queen_delta]
xlabels = ["Combined","Beatles", "RWC_pop","RW","Queen"]

fig,ax =fig, ax = plt.subplots(figsize=(5.5, 2.5))
violin_parts = ax.violinplot(data,showmeans=False, showmedians=True,
        showextrema=False)
violin_parts["cmedians"].set_color("red")
for x,color in zip(violin_parts["bodies"],['r','g','b','purple']):
    x.set_color(color)
bplot_parts = ax.boxplot(data,
            showfliers=True,medianprops=dict(linestyle=None,linewidth=0),
            flierprops=dict(markerfacecolor='k', marker='o',markersize=1),
            widths=0.1)
ax.set_yticks(np.arange(-10,15,5))
ax.set_ylim(-15,15)
ax.set_ylabel("Delta F-score in %")
ax.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
ax.set_xlim(0.5, len(xlabels) + 0.5);
ax.grid("on")
fig.tight_layout(pad=0.1)
fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/median_rp_dataset.png")


delta_f = [x-y for x,y in zip(majmin_f_rp_sev,majmin_f_sev)]
beatles_delta = [x-y for x,y in zip(beatles_f_rp_sev,beatles_f_sev)]
rwc_delta = [x-y for x,y in zip(rwc_f_rp_sev,rwc_f_sev)]
rw_delta = [x-y for x,y in zip(rw_f_rp_sev,rw_f_sev)]
queen_delta = [x-y for x,y in zip(queen_f_rp_sev,queen_f_sev)]
data = [delta_f,beatles_delta,rwc_delta,rw_delta,queen_delta]
xlabels = ["Combined","Beatles", "RWC_pop","RW","Queen"]

fig,ax =fig, ax = plt.subplots(figsize=(5.5, 2.5))
violin_parts = ax.violinplot(data,showmeans=False, showmedians=True,
        showextrema=False)
violin_parts["cmedians"].set_color("red")
for x,color in zip(violin_parts["bodies"],['r','g','b','purple']):
    x.set_color(color)
bplot_parts = ax.boxplot(data,
            showfliers=True,medianprops=dict(linestyle=None,linewidth=0),
            flierprops=dict(markerfacecolor='k', marker='o',markersize=1),
            widths=0.1)
ax.set_yticks(np.arange(-10,15,5))
ax.set_ylim(-15,15)
ax.set_ylabel("Delta F-score in %")
ax.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
ax.set_xlim(0.5, len(xlabels) + 0.5);
ax.grid("on")
fig.tight_layout(pad=0.1)
# fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/median_rp_dataset_sevenths.png")


# Wilcoxon test
A Wilcoxon signed-rank test was calculated to examine the effect of using a recurrency plot to smooth the chromagram instead of horizontal median filters. The transcription result measured are compared with the F-scores using the majmin alphabet. The distribution of the paired differences was symmetrical following visual inspection of the histogram. There was a statistically significant increase in F-scores (Mdn = 77.0%) using the recurrency plot compared to the use of median filters (Mdn = 75.5%), t = 12175.0, p < .001

In [ ]:
diff = [x - y for x,y in zip(majmin_f_rp, majmin_f_median)]
diff_sorted = sorted([(y,x-y) for x,y in zip(majmin_f_rp, majmin_f_median)],key=lambda x:x[0])

diff_sevenths = [x - y for x,y in zip(sevenths_f_rp,sevenths_f_median)]
fig,ax = plt.subplots(figsize=(7,3))
ax.grid(True)
ax.hist(diff_sevenths,bins=np.linspace(-15,15,30))
ax.set_ylabel("histogram count")
ax.set_xlabel("paired difference majmin score")
ax.set_title("RP - Median")
res = stats.wilcoxon(diff,method='approx')
res_sevenths = stats.wilcoxon(diff_sevenths,method='approx')

print("Wilcoxon test for paired samples")
print(f"Majmin: z={res.zstatistic:0.1f}, p={res.pvalue:0.3f}, r={np.abs(res.zstatistic) / np.sqrt(len(diff)):0.2f}")
print(f"Sevenths: z={res_sevenths.zstatistic:0.1f}, p={res_sevenths.pvalue:0.3f}, r={np.abs(res_sevenths.zstatistic) / np.sqrt(len(diff)):0.2f}")


fig, ax1 = plt.subplots(figsize=(3, 3.5))
parts = ax1.violinplot([diff,diff_sevenths],showmeans=False, showmedians=True,
        showextrema=False)
bplot = ax1.boxplot([diff,diff_sevenths],widths=0.1,showfliers=True,medianprops=dict(linestyle=None,linewidth=0),flierprops=dict(markerfacecolor='k', marker='o',markersize=1))
parts["cmedians"].set_color("red")
xlabels = ["majmin","sevenths"]
ax1.set_yticks(np.arange(-20,30,5))
ax1.set_ylim(-15,15)
ax1.set_ylabel("Delta F-score in %")
ax1.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
ax1.set_xlim(0.5, len(xlabels) + 0.5);
ax1.grid("on")
fig.tight_layout(pad=0.1)
fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/median_rp_delta.png")

# sort results by f-scores in ascending order
majmin_sorted = [x[0] for x in diff_sorted]
delta = [x[1] for x in diff_sorted]
fig,(ax0,ax1) = plt.subplots(2,1,figsize=(9,6),sharex=True)
ax0.plot(delta)
ax0.grid("on")
ax0.set_ylim(-25,25)
ax0.set_ylabel("Delta F-score in %")
ax1.plot(majmin_sorted)
ax1.set_ylabel("F-score in %")
ax1.set_xlim(0,350)
ax1.set_xlabel("Track number")
ax1.grid("on")
fig.tight_layout(pad=0.1)
fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/median_rp_sorted_delta.png")


### Study on source separation

In [ ]:
results = {}
for source_sep in ["none","vocals","drums","both"]:
        results[source_sep], datasets = load_results(f"/home/max/ET-TI/Masterarbeit/results/median_{source_sep}")

results_rp,_ = load_results("/home/max/ET-TI/Masterarbeit/results/rp_none")
# list of tracks that should be excluded due to issues with the annotations / or lack of harmonic content
outliers = ['03_-_Anna_(Go_To_Him)', # tuning issues
            '10_-_Lovely_Rita', # tuning issues
            'CD1_-_05_-_Wild_Honey_Pie', # little harmonic content
            'CD1_-_06_-_The_Continuing_Story_of_Bungalow_Bill', # faulty audio or annotations
            'CD2_-_12_-_Revolution_9',  # little harmonic content
            '08_-_Within_You_Without_You', # leslie effect on piano => bad chroma
            '02 Another One Bites The Dust', # little harmonic content
            '16 We Will Rock You', # little harmonic content
            "Stalker's Day Off (I've Been Hanging Around)", # faulty audio / issues with annotations
            'Stand Your Ground'  # faulty audio / issues with annotations
]

majmin_none = [100*x.majmin_f for x in results["none"] if x.name not in outliers]
majmin_rp = [100*x.majmin_f for x in results_rp if x.name not in outliers]
majmin_vocals = [100*x.majmin_f for x in results["vocals"] if x.name not in outliers]
majmin_drums = [100*x.majmin_f for x in results["drums"] if x.name not in outliers]
majmin_both = [100*x.majmin_f for x in results["both"] if x.name not in outliers]
sevenths_none = [100*x.sevenths_f for x in results["none"] if x.name not in outliers]
sevenths_vocals = [100*x.sevenths_f for x in results["vocals"] if x.name not in outliers]
sevenths_drums = [100*x.sevenths_f for x in results["drums"] if x.name not in outliers]
sevenths_both = [100*x.sevenths_f for x in results["both"] if x.name not in outliers]

delta_median_rp = [x-y for x,y in zip(majmin_f_rp, majmin_f_median)]
delta_majmin_vocals = [y-x for x,y in zip(majmin_none,majmin_vocals)]
delta_majmin_drums = [y-x for x,y in zip(majmin_none,majmin_drums)]
delta_majmin_both = [y-x for x,y in zip(majmin_none,majmin_both)]

fig, ax1 = plt.subplots(figsize=(4, 4))
parts = ax1.violinplot([delta_majmin_vocals,delta_majmin_drums,delta_majmin_both],showmeans=False, showmedians=True,
        showextrema=False)
bplot = ax1.boxplot([delta_majmin_vocals,delta_majmin_drums,delta_majmin_both],
                    showfliers=True,medianprops=dict(linestyle=None,linewidth=0),
                    flierprops=dict(markerfacecolor='k', marker='o',markersize=1),
                    widths=0.1)
parts["cmedians"].set_color("red")
xlabels = ["no vocals","no drums","no drums/vocals"]
ax1.set_yticks(np.arange(-35,35,5))
ax1.set_ylim(-30,30)
ax1.set_ylabel("Delta F-score in %")
ax1.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
ax1.set_xlim(0.5, len(xlabels) + 0.5);
ax1.grid("on")
fig.tight_layout(pad=0.1)
fig.savefig("/home/max/ET-TI/Masterarbeit/latex/figures/source_separation_majmin_result.png")

delta_drums_vocals = [y-x for x,y in zip(majmin_both,majmin_drums)]
print("Majmin")
res_v = stats.wilcoxon(delta_majmin_vocals,method='approx')
res_d = stats.wilcoxon(delta_majmin_drums,method='approx')
res_h = stats.wilcoxon(delta_majmin_both,method='approx')
res_dv = stats.wilcoxon(delta_drums_vocals,method='approx')

print("Wilcoxon test for paired samples")
print(f"comparison mix/no vocals: N={len(delta_majmin_vocals)}, z={res_v.zstatistic:0.1f}, p={res_v.pvalue:0.3f}, r={np.abs(res_v.zstatistic) / np.sqrt(len(delta_majmin_vocals)):0.2f}")
print(f"comparison mix/no drums: N={len(delta_majmin_vocals)}, z={res_d.zstatistic:0.1f}, p={res_d.pvalue:0.3f}, r={np.abs(res_d.zstatistic) / np.sqrt(len(delta_majmin_vocals)):0.2f}")
print(f"comparison mix/no vocals and drums: N={len(delta_majmin_vocals)}, z={res_h.zstatistic:0.1f}, p={res_h.pvalue:0.3f}, r={np.abs(res_h.zstatistic) / np.sqrt(len(delta_majmin_vocals)):0.2f}")
print(f"comparison no drums / no drums and no vocals: N={len(delta_majmin_vocals)}, z={res_dv.zstatistic:0.1f}, p={res_dv.pvalue:0.3f}, r={np.abs(res_dv.zstatistic) / np.sqrt(len(delta_majmin_vocals)):0.2f}")


In [ ]:
## Dataset specific eximination
separation_method = "both"

f_none = [100*x.majmin_f for x in results["none"] if x.name not in outliers]
beatles_f_none = [100*x.majmin_f for x in results["none"] if x.name not in outliers and x.dataset == "beatles"]
rwc_f_none = [100*x.majmin_f for x in results["none"] if x.name not in outliers and x.dataset == "rwc_pop"]
rw_f_none = [100*x.majmin_f for x in results["none"] if x.name not in outliers and x.dataset == "rw"]
queen_f_none = [100*x.majmin_f for x in results["none"] if x.name not in outliers and x.dataset == "queen"]

f_separated = [100*x.majmin_f for x in results[separation_method] if x.name not in outliers]
beatles_f = [100*x.majmin_f for x in results[separation_method] if x.name not in outliers and x.dataset == "beatles"]
rwc_f = [100*x.majmin_f for x in results[separation_method] if x.name not in outliers and x.dataset == "rwc_pop"]
rw_f = [100*x.majmin_f for x in results[separation_method] if x.name not in outliers and x.dataset == "rw"]
queen_f = [100*x.majmin_f for x in results[separation_method] if x.name not in outliers and x.dataset == "queen"]

delta = [x-y for x,y in zip(f_separated,f_none)]
beatles_delta = [x-y for x,y in zip(beatles_f,beatles_f_none)]
rwc_delta = [x-y for x,y in zip(rwc_f,rwc_f_none)]
rw_delta = [x-y for x,y in zip(rw_f,rw_f_none)]
queen_delta = [x-y for x,y in zip(queen_f,queen_f_none)]

fig, ax1 = plt.subplots(figsize=(5.5, 2.5))
parts = ax1.violinplot([delta,beatles_delta, rwc_delta, rw_delta, queen_delta],showmeans=False, showmedians=True,
        showextrema=False)
bplot = ax1.boxplot([delta,beatles_delta, rwc_delta, rw_delta, queen_delta],
                    showfliers=True,medianprops=dict(linestyle=None,linewidth=0),
                    flierprops=dict(markerfacecolor='k', marker='o',markersize=1),
                    widths=0.1)
parts["cmedians"].set_color("red")
for x,color in zip(parts["bodies"],['r','g','b','purple']):
    x.set_color(color)
xlabels = ["Combined","Beatles","RWC","RW","Queen"]
ax1.set_yticks(np.arange(-20,25,5))
ax1.set_ylim(-25,25)
ax1.set_ylabel("Delta F-score in %")
ax1.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
ax1.set_xlim(0.5, len(xlabels) + 0.5);
ax1.grid("on")
fig.tight_layout(pad=0.1)


med_seg_sevenths = 100 * np.median(beatles_f)
iqr_seg_sevenths = 100 * np.subtract(*np.percentile(beatles_f, [75, 25]))

fig.savefig(f"/home/max/ET-TI/Masterarbeit/latex/figures/separation_{separation_method}_delta.png")
print(f"Majmin: separation {separation_method}")
res_beatles = stats.wilcoxon(beatles_delta,method='approx')
res_rwc = stats.wilcoxon(rwc_delta,method='approx')
res_rw = stats.wilcoxon(rw_delta,method='approx')
res_queen = stats.wilcoxon(queen_delta,method='approx')
print("Wilcoxon test for paired samples")
print(f"beatles: N={len(beatles_delta)} z={res_beatles.zstatistic:0.1f}, p={res_beatles.pvalue:0.3f}, r={np.abs(res_beatles.zstatistic) / np.sqrt(len(beatles_delta)):0.2f}")
print(f"median: {np.median(beatles_f)} +- {np.subtract(*np.percentile(beatles_f, [75, 25]))}")
print(f"rwc: N={len(rwc_delta)}, z={res_rwc.zstatistic:0.1f}, p={res_rwc.pvalue:0.3f}, r={np.abs(res_rwc.zstatistic) / np.sqrt(len(rwc_delta)):0.2f}")
print(f"median: {np.median(rwc_f)} +- {np.subtract(*np.percentile(rwc_f, [75, 25]))}")
print(f"rw: N={len(rw_delta)}, z={res_rw.zstatistic:0.1f}, p={res_rw.pvalue:0.3f}, r={np.abs(res_rw.zstatistic) / np.sqrt(len(rw_delta)):0.2f}")
print(f"median: {np.median(rw_f)} +- {np.subtract(*np.percentile(rw_f, [75, 25]))}")
print(f"queen: N={len(queen_delta)}, z={res_queen.zstatistic:0.1f}, p={res_queen.pvalue:0.3f}, r={np.abs(res_queen.zstatistic) / np.sqrt(len(queen_delta)):0.2f}")
print(f"median: {np.median(queen_f)} +- {np.subtract(*np.percentile(queen_f, [75, 25]))}")

#### Study on correlation vs inner product

In [ ]:
from visualization import load_results
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

results_inner,_ = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/median_drums.hdf5")
results_corr,_ = load_results("/home/max/ET-TI/Masterarbeit/chord-recognition/results/correlation_median_drums.hdf5")
# list of tracks that should be excluded due to issues with the annotations / or lack of harmonic content
outliers = ['03_-_Anna_(Go_To_Him)', # tuning issues
            '10_-_Lovely_Rita', # tuning issues
            'CD1_-_05_-_Wild_Honey_Pie', # little harmonic content
            'CD1_-_06_-_The_Continuing_Story_of_Bungalow_Bill', # faulty audio or annotations
            'CD2_-_12_-_Revolution_9',  # little harmonic content
            '08_-_Within_You_Without_You', # leslie effect on piano => bad chroma
            '02 Another One Bites The Dust', # little harmonic content
            '16 We Will Rock You', # little harmonic content
            "Stalker's Day Off (I've Been Hanging Around)", # faulty audio / issues with annotations
            'Stand Your Ground'  # faulty audio / issues with annotations
]

majmin_inner = [100*x.majmin_f for x in results_inner if x.name not in outliers]
majmin_corr = [100*x.majmin_f for x in results_corr if x.name not in outliers]

sevenths_inner = [x for x in results_inner if x.name not in outliers]
sevenths_corr = [x for x in results_corr if x.name not in outliers]


f_none = [100*x.sevenths_f for x in results_inner if x.name not in outliers]
beatles_f_none = [100*x.sevenths_f for x in results_inner if x.name not in outliers and x.dataset == "beatles"]
rwc_f_none = [100*x.sevenths_f for x in results_inner if x.name not in outliers and x.dataset == "rwc_pop"]
rw_f_none = [100*x.sevenths_f for x in results_inner if x.name not in outliers and x.dataset == "rw"]
queen_f_none = [100*x.sevenths_f for x in results_inner if x.name not in outliers and x.dataset == "queen"]

f_separated = [100*x.sevenths_f for x in results_corr if x.name not in outliers]
beatles_f = [100*x.sevenths_f for x in results_corr if x.name not in outliers and x.dataset == "beatles"]
rwc_f = [100*x.sevenths_f for x in results_corr if x.name not in outliers and x.dataset == "rwc_pop"]
rw_f = [100*x.sevenths_f for x in results_corr if x.name not in outliers and x.dataset == "rw"]
queen_f = [100*x.sevenths_f for x in results_corr if x.name not in outliers and x.dataset == "queen"]

delta = [x-y for x,y in zip(f_separated,f_none)]
beatles_delta = [x-y for x,y in zip(beatles_f,beatles_f_none)]
rwc_delta = [x-y for x,y in zip(rwc_f,rwc_f_none)]
rw_delta = [x-y for x,y in zip(rw_f,rw_f_none)]
queen_delta = [x-y for x,y in zip(queen_f,queen_f_none)]
delta_sorted = sorted([(y.majmin_f,100 * (x.sevenths_f-y.sevenths_f),x.name,y.name) for x,y in zip(sevenths_corr, sevenths_inner)],key=lambda x:x[0])


fig, ax1 = plt.subplots(figsize=(5.5, 2.5))
parts = ax1.violinplot([delta,beatles_delta, rwc_delta, rw_delta, queen_delta],showmeans=False, showmedians=True,
        showextrema=False)
bplot = ax1.boxplot([delta,beatles_delta, rwc_delta, rw_delta, queen_delta],
                    showfliers=True,medianprops=dict(linestyle=None,linewidth=0),
                    flierprops=dict(markerfacecolor='k', marker='o',markersize=1),
                    widths=0.1)
parts["cmedians"].set_color("red")
for x,color in zip(parts["bodies"],['r','g','b','purple']):
    x.set_color(color)
xlabels = ["Combined","Beatles","RWC","RW","Queen"]
ax1.set_yticks(np.arange(-35,40,5))
ax1.set_ylim(-35,35)
ax1.set_ylabel("Delta F-score in %")
ax1.set_xticks(np.arange(1, len(xlabels) + 1), labels=xlabels)
ax1.set_xlim(0.5, len(xlabels) + 0.5);
ax1.grid("on")
fig.tight_layout(pad=0.1)
fig.savefig(f"/home/max/ET-TI/Masterarbeit/latex/figures/correlation_delta.png")

print(f"Majmin: corr")
res = stats.wilcoxon(delta,method='approx')
res_beatles = stats.wilcoxon(beatles_delta,method='approx')
res_rwc = stats.wilcoxon(rwc_delta,method='approx')
res_rw = stats.wilcoxon(rw_delta,method='approx')
res_queen = stats.wilcoxon(queen_delta,method='approx')
print("Wilcoxon test for paired samples")
print(f"combined: N={len(delta)} z={res.zstatistic:0.1f}, p={res.pvalue:0.3f}, r={np.abs(res.zstatistic) / np.sqrt(len(delta)):0.2f}")
print(f"beatles: N={len(beatles_delta)} z={res_beatles.zstatistic:0.1f}, p={res_beatles.pvalue:0.3f}, r={np.abs(res_beatles.zstatistic) / np.sqrt(len(beatles_delta)):0.2f}")
print(f"rwc: N={len(rwc_delta)}, z={res_rwc.zstatistic:0.1f}, p={res_rwc.pvalue:0.3f}, r={np.abs(res_rwc.zstatistic) / np.sqrt(len(rwc_delta)):0.2f}")
print(f"rw: N={len(rw_delta)}, z={res_rw.zstatistic:0.1f}, p={res_rw.pvalue:0.3f}, r={np.abs(res_rw.zstatistic) / np.sqrt(len(rw_delta)):0.2f}")
print(f"queen: N={len(queen_delta)}, z={res_queen.zstatistic:0.1f}, p={res_queen.pvalue:0.3f}, r={np.abs(res_queen.zstatistic) / np.sqrt(len(queen_delta)):0.2f}")

majmin_sorted = [x[0] for x in delta_sorted]
delta = [x[1] for x in delta_sorted]
fig,(ax0,ax1) = plt.subplots(2,1,figsize=(7,5),sharex=True)
ax0.plot(delta)
ax0.grid("on")
ax0.set_ylim(-45,45)
ax0.set_ylabel("Delta F-score in %")
ax1.plot(majmin_sorted)
ax1.set_ylabel("F-score in %")
ax1.set_xlim(0,350)
ax1.set_xlabel("Track number")
ax1.grid("on")
fig.tight_layout(pad=0.1)
fig.savefig(f"/home/max/ET-TI/Masterarbeit/latex/figures/correlation_delta_sorted.png")

In [ ]:
from visualization import load_results
import numpy as np

def createTable(filepath): 
    """loads the complete results for the dataset"""
    # Function to format a row
    def format_row(dset, f, wcsr, q):
        return f"{dset} & {100*np.median(f):0.1f} $\\pm$ {100*np.subtract(*np.percentile(f, [75, 25])):0.1f} \
            & {100*np.median(wcsr):0.1f} $\\pm$ {100*np.subtract(*np.percentile(wcsr, [75, 25])):0.1f} \
                & {100*np.median(q):0.1f} $\\pm$ {100*np.subtract(*np.percentile(q, [75, 25])):0.1f} \\\\"
    results,datasets = load_results(filepath)
    outliers = ['03_-_Anna_(Go_To_Him)', # tuning issues
            '10_-_Lovely_Rita', # tuning issues
            'CD1_-_05_-_Wild_Honey_Pie', # little harmonic content
            'CD1_-_06_-_The_Continuing_Story_of_Bungalow_Bill', # faulty audio or annotations
            'CD2_-_12_-_Revolution_9',  # little harmonic content
            '08_-_Within_You_Without_You', # leslie effect on piano => bad chroma
            '02 Another One Bites The Dust', # little harmonic content
            '16 We Will Rock You', # little harmonic content
            "Stalker's Day Off (I've Been Hanging Around)", # faulty audio / issues with annotations
            'Stand Your Ground'  # faulty audio / issues with annotations
    ]

    print("\\begin{tabular}{lcccc}\n\\toprule\nDataset & F-score in \% & WCSR in \%& Q in \% \\\\\n\\midrule")
    majmin_f = [x.majmin_f for x in results if x.name not in outliers]
    majmin_wcsr = [x.majmin_wscr for x in results if x.name not in outliers]
    majmin_q = [x.majmin_seg for x in results if x.name not in outliers]
    sevenths_f = [x.sevenths_f for x in results if x.name not in outliers]
    sevenths_wcsr = [x.sevenths_wscr for x in results if x.name not in outliers]
    sevenths_q = [x.sevenths_seg for x in results if x.name not in outliers]
    print(format_row("Combined", majmin_f, majmin_wcsr, majmin_q))
    print(format_row("Combined", sevenths_f, sevenths_wcsr, sevenths_q))
    print("\midrule")
    for dset in datasets: 
        majmin_f = [x.majmin_f for x in results if x.name not in outliers and x.dataset == dset]
        majmin_wcsr = [x.majmin_wscr for x in results if x.name not in outliers and x.dataset == dset]
        majmin_q = [x.majmin_seg for x in results if x.name not in outliers and x.dataset == dset]
        sevenths_f = [x.sevenths_f for x in results if x.name not in outliers]
        sevenths_wcsr = [x.sevenths_wscr for x in results if x.name not in outliers]
        sevenths_q = [x.sevenths_seg for x in results if x.name not in outliers]
        print(format_row(dset, majmin_f, majmin_wcsr, majmin_q))
        print(format_row(dset, sevenths_f, sevenths_wcsr, sevenths_q))
        print("\midrule")
    print("\\bottomrule\n\\end{tabular}")    



createTable("/home/max/ET-TI/Masterarbeit/chord-recognition/results/cpss.hdf5")
